# Downloading and Transforming the Taxi Trip Data

In [1]:
# Libraries
import pandas as pd
import numpy as np
import geopandas
import requests

from geopandas.tools import sjoin
from datetime import datetime

In [2]:
path = ''

In [ ]:
# Downloading
for y in range(2009, 2021):
    for m in range(1, 13):
        if m // 10 == 0:
            m = str(0)+str(m)
            
        url = 'https://nyc-tlc.s3.amazonaws.com/trip+data/yellow_tripdata_'+str(y)+'-'+str(m)+'.csv'
        r = requests.get(url, allow_redirects=True)
        name = path+'Raw_data/tripdata_'+str(y)+'-'+str(m)+'.csv'
        open(name, 'wb').write(r.content)
        print(url)

In [ ]:
# Transforming
taxi_zones = geopandas.read_file(path+'Raw_data/taxi_zones/taxi_zones.shp')
taxi_zones = taxi_zones.to_crs(epsg = 4326)
taxi_zones = taxi_zones.loc[:, ['LocationID', 'geometry']]

for y in range(2009, 2021):
    df_complete = pd.DataFrame()
    print(y, datetime.now().strftime('%H:%M'))

    for m in range(1, 13):
        print(m, datetime.now().strftime('%H:%M'))
        if m // 10 == 0:
            m = str(0)+str(m)

        path = 'Raw_data/tripdata_'+str(y)+'-'+str(m)+'.csv'

        if (y < 2016) | ((y<2017) & (int(m)<7)):
            df = pd.read_csv(path, usecols = ['Trip_Pickup_DateTime', 
                                              'Trip_Dropoff_DateTime', 
                                              'Trip_Distance', 
                                              'Start_Lon', 
                                              'Start_Lat'], 
                             low_memory = False)

            df['Trip_Pickup_DateTime']=pd.to_datetime(df['Trip_Pickup_DateTime'], format='%Y-%m-%d %H:%M:%S')
            df['Trip_Dropoff_DateTime']=pd.to_datetime(df['Trip_Dropoff_DateTime'], format='%Y-%m-%d %H:%M:%S')
            df['trip_duration'] = (df['Trip_Dropoff_DateTime']-df['Trip_Pickup_DateTime']) / np.timedelta64(1, 'm')

            df['pickup_year'] = df['Trip_Pickup_DateTime'].dt.year
            df['pickup_month'] = df['Trip_Pickup_DateTime'].dt.month
            df['pickup_week'] = df['Trip_Pickup_DateTime'].dt.isocalendar().week
            df['pickup_day'] = df['Trip_Pickup_DateTime'].dt.day
            df['pickup_day_of_week'] = df['Trip_Pickup_DateTime'].dt.weekday
            df['pickup_hour'] = df['Trip_Pickup_DateTime'].dt.hour

            df_clean = df[(df['Trip_Distance'] > 0) & (df['trip_duration'] > 0) & (df['pickup_year']==y) & (df['pickup_month']==int(m))]
            print('df clean created', datetime.now().strftime('%H:%M'))

            points = geopandas.GeoDataFrame(df_clean.loc[:, ['pickup_year', 
                                                             'pickup_month', 
                                                             'pickup_week', 
                                                             'pickup_day', 
                                                             'pickup_hour', 
                                                             'pickup_day_of_week']], 
                                            crs = {'init': 'epsg:4326', 'no_defs': True},
                                            geometry = geopandas.points_from_xy(df_clean['Start_Lon'], 
                                                                                df_clean['Start_Lat']))
            print('point created', datetime.now().strftime('%H:%M'))

            df_with_zone = sjoin(points, taxi_zones, how = 'inner', op = 'intersects')
            
            del df_with_zone['geometry']
            del df_with_zone['index_right']
            df_final = df_with_zone.rename(columns = {'LocationID':'PULocationID'})
            print('df final created', datetime.now().strftime('%H:%M'))       
            
        if ((y==2016) & (int(m) > 6)):
            df = pd.read_csv(path, low_memory = False)
            df.reset_index(inplace=True)
            df.columns = ['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime', 'passenger_count', 
                          'trip_distance', 'RatecodeID', 'store_and_fwd_flag', 
                          'PULocationID', 'DOLocationID',    'payment_type', 'fare_amount', 'extra', 
                          'mta_tax', 'tip_amount', 'tolls_amount', 
                          'improvement_surcharge', 'total_amount', 'level_0', 'level_1']

            df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
            df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
            df['trip_duration'] = (df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']) / np.timedelta64(1, 'm')

            df['pickup_year'] = df['tpep_pickup_datetime'].dt.year
            df['pickup_month'] = df['tpep_pickup_datetime'].dt.month
            df['pickup_week'] = df['tpep_pickup_datetime'].dt.isocalendar().week
            df['pickup_day'] = df['tpep_pickup_datetime'].dt.day
            df['pickup_day_of_week'] = df['tpep_pickup_datetime'].dt.weekday
            df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

            df_clean = df[(df['trip_distance'] > 0) & (df['trip_duration'] > 0) & (df['pickup_year']==y) & (df['pickup_month']==int(m))]
            df_final = df_clean.loc[:, ['PULocationID', 'pickup_year', 'pickup_month', 
                                        'pickup_week', 'pickup_day', 'pickup_hour', 'pickup_day_of_week']]
        
        else:
            df = pd.read_csv(path, usecols = ['tpep_pickup_datetime', 'tpep_dropoff_datetime', 
                                              'PULocationID', 'trip_distance'], low_memory = False)

            df['tpep_pickup_datetime']=pd.to_datetime(df['tpep_pickup_datetime'], format='%Y-%m-%d %H:%M:%S')
            df['tpep_dropoff_datetime']=pd.to_datetime(df['tpep_dropoff_datetime'], format='%Y-%m-%d %H:%M:%S')
            df['trip_duration'] = (df['tpep_dropoff_datetime']-df['tpep_pickup_datetime']) / np.timedelta64(1, 'm')

            df['pickup_year'] = df['tpep_pickup_datetime'].dt.year
            df['pickup_month'] = df['tpep_pickup_datetime'].dt.month
            df['pickup_week'] = df['tpep_pickup_datetime'].dt.isocalendar().week
            df['pickup_day'] = df['tpep_pickup_datetime'].dt.day
            df['pickup_day_of_week'] = df['tpep_pickup_datetime'].dt.weekday
            df['pickup_hour'] = df['tpep_pickup_datetime'].dt.hour

            df_clean = df[(df['trip_distance'] > 0) & (df['trip_duration'] > 0) & (df['pickup_year']==y) & (df['pickup_month']==int(m))]
            df_final = df_clean.loc[:, ['PULocationID', 'pickup_year', 'pickup_month', 
                                        'pickup_week', 'pickup_day', 'pickup_hour', 'pickup_day_of_week']]

        df_agg = df_final.groupby(['pickup_year', 'pickup_month', 'pickup_week','pickup_day', 
                                   'pickup_day_of_week','pickup_hour', 'PULocationID']).size().reset_index()
        df_agg = df_agg.rename(columns = {0:'count'})
        print('df agg created', datetime.now().strftime('%H:%M'))

        df_complete = df_complete.append(df_agg, ignore_index=True)
        print('df complete created', datetime.now().strftime('%H:%M'))
        
        name = path+'Raw_data/tripdata_'+str(y)+'-'+str(m)+'.csv'
        df_complete.to_csv(name)
        print(y, m, 'saved')

In [ ]:
# Yearly Datasets
for y in range(2009, 2021):
    print(y)
    df_year_complete = pd.DataFrame()

    for m in range(1, 13):
        if m < 10:
        m = str(0)+str(m)

    file = path+'Raw_data/tripdata_'+str(y)+'-'+str(m)+'.csv'
    df = pd.read_csv(file, usecols = ['pickup_year', 'pickup_month', 'pickup_week', 'pickup_day', 
                                      'pickup_day_of_week', 'pickup_hour', 'PULocationID', 'count'])

    df_year_complete = df_year_complete.append(df)

name = path+'Data/tripdata_'+str(y)+'.csv'
df_year_complete.to_csv(name)

In [ ]:
# Complete Dataset
df_complete = pd.DataFrame()
for y in range(2009, 2021):
    print(y)
    file = path+'Data/tripdata_'+str(y)+'.csv'
    df = pd.read_csv(file, usecols = ['pickup_year', 'pickup_month', 'pickup_week', 'pickup_day', 
                                      'pickup_day_of_week', 'pickup_hour', 'PULocationID', 'count'])
    df_complete = df_complete.append(df)
    name = path+'Data/tripdata.csv'
    df_complete.to_csv(name)  

In [ ]:
# Dataset with top 20 taxi zones only
zones_top20 = df_complete.groupby('PULocationID')['count'].sum().sort_values(ascending = False).head(21).sort_index()
zones_top20 = zones_top20.drop(138) # Top 20 zones in Manhatten

taxi_zones.loc[taxi_zones['LocationID'].isin(zones_top20.index), 'top20_MH'] = 'Top 20 Manhatten'
taxi_zones.loc[~taxi_zones['LocationID'].isin(zones_top20.index), 'top20_MH'] = 'Other'

taxi_zones_top20 = taxi_zones[taxi_zones['top20_MH']=='Top 20 Manhatten']
taxi_zones_top20['count'] = zones_top20.values

df_top20 = df[df['PULocationID'].isin(zones_top20.index)]
name = path+'Data/df_top20.csv'
df_top20.to_csv(name) 